## Este notebook descarga imágenes de Sentinel 2 para las zonas de alertas de GFW


In [3]:
# Sentinel-2 Download for Alert Points

import geopandas as gpd
from sentinelsat import SentinelAPI, geojson_to_wkt
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()

True

In [14]:
os.chdir('/Users/Daniel/Desktop/bosques_bog/bosques-bog/')

In [15]:
geojson_path = os.path.join('temp_data', 'alertas_gfw.geojson')
alerts_gfw = gpd.read_file(geojson_path)
alerts_gfw.head()

,latitude,longitude,gfw_integrated_alerts__date,gfw_integrated_alerts__confidence,umd_glad_landsat_alerts__confidence,umd_glad_sentinel2_alerts__confidence,wur_radd_alerts__confidence,geometry
0,4.99125,-74.00775,2024-01-05,high,high,not_detected,not_detected,POINT (-74.00775 4.99125)
1,4.99125,-74.00765,2024-01-05,high,high,not_detected,not_detected,POINT (-74.00765 4.99125)
2,4.99125,-74.00755,2024-01-05,high,high,not_detected,not_detected,POINT (-74.00755 4.99125)
3,4.99125,-74.00745,2024-01-05,high,high,not_detected,not_detected,POINT (-74.00745 4.99125)
4,4.99125,-74.00735,2024-01-05,high,high,not_detected,not_detected,POINT (-74.00735 4.99125)


In [16]:

COPERNICUS_USERID = os.getenv("COPERNICUS_USERID")
COPERNICUS_PASSWORD = os.getenv("COPERNICUS_PASSWORD")

In [17]:
# Sentinel API credentials (replace with your own)
api = SentinelAPI(COPERNICUS_USERID, COPERNICUS_PASSWORD, 'https://apihub.copernicus.eu/apihub')

In [19]:

# Download folder
download_dir = 'temp_data'

# Loop through alerts and download Sentinel-2 images
for idx, row in alerts_gfw.iterrows():
    # Create a small buffer (e.g., 100m) around the alert point
    geom = row.geometry.buffer(0.001)  # ~100m buffer, adjust as needed
    footprint = geojson_to_wkt(geom.__geo_interface__)
    
    # Use alert date +/- 3 days
    alert_date = row.get('gfw_integrated_alerts__date', None)
    if alert_date is not None:
        if hasattr(alert_date, 'to_pydatetime'):
            date_obj = alert_date.to_pydatetime()
        elif isinstance(alert_date, str):
            date_obj = datetime.strptime(alert_date, "%Y-%m-%d")
        else:
            date_obj = alert_date  # fallback, if already datetime
        date_start = (date_obj - timedelta(days=3)).strftime("%Y%m%d")
        date_end = (date_obj + timedelta(days=3)).strftime("%Y%m%d")
    else:
        # Default to last month if no date
        date_start = (datetime.now() - timedelta(days=30)).strftime("%Y%m%d")
        date_end = datetime.now().strftime("%Y%m%d")
    
    # Query Sentinel-2 L2A products
    products = api.query(
        footprint,
        date=(date_start, date_end),
        platformname='Sentinel-2',
        processinglevel='Level-2A',
        cloudcoverpercentage=(0, 30)
    )
    
    # Download first found product (or skip if none)
    if products:
        uuid = list(products.keys())[0]
        print(f"Downloading product {uuid} for alert {idx}")
        api.download(uuid, directory_path=download_dir)
    else:
        print(f"No product found for alert {idx}")

# Note: For large numbers of alerts, consider deduplicating queries or using batch download.

ConnectionError: HTTPSConnectionPool(host='apihub.copernicus.eu', port=443): Max retries exceeded with url: /apihub/search?format=json&rows=100&start=0&q=beginPosition%3A%5B%222024-01-02T00%3A00%3A00Z%22+TO+%222024-01-08T00%3A00%3A00Z%22%5D+cloudcoverpercentage%3A%5B%220%22+TO+%2230%22%5D+platformname%3A%22Sentinel-2%22+processinglevel%3A%22Level-2A%22+footprint%3A%22Intersects%28POLYGON%28%28-74.0067+4.9912%2C-74.0068+4.9912%2C-74.0068+4.9911%2C-74.0068+4.9910%2C-74.0068+4.9909%2C-74.0069+4.9908%2C-74.0069+4.9907%2C-74.0070+4.9906%2C-74.0070+4.9905%2C-74.0071+4.9905%2C-74.0072+4.9904%2C-74.0073+4.9904%2C-74.0074+4.9903%2C-74.0075+4.9903%2C-74.0076+4.9903%2C-74.0077+4.9903%2C-74.0078+4.9902%2C-74.0078+4.9903%2C-74.0079+4.9903%2C-74.0080+4.9903%2C-74.0081+4.9903%2C-74.0082+4.9904%2C-74.0083+4.9904%2C-74.0084+4.9905%2C-74.0085+4.9905%2C-74.0085+4.9906%2C-74.0086+4.9907%2C-74.0086+4.9908%2C-74.0087+4.9909%2C-74.0087+4.9910%2C-74.0087+4.9911%2C-74.0087+4.9912%2C-74.0088+4.9912%2C-74.0087+4